In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Comparing PWA Results With New Event Selection
We've updated our event selection using FSRoot and a handful of new cuts, reducing our statistics, and potentially affecting the kinematics of our events such that the PWA could substantially change. Here we will compare the PWA results using this new event selection to our results obtained from our old event selection that has much looser cuts. The same model will be used:
* Only spin-1 ($J=1$) waves
  * {$J^P\ell$} = {$1^+S, 1^-P, 1^+D$}
* The D and S waves are constrained to each other, such that every $D$-wave has a common ratio and phase between its $S$-wave partner
  * Boundaries: $0< r_{D/S} < 1,\quad -\pi<\phi_{D-S}<\pi$
* Include an isotropic background term
  * Assumed that sideband subtraction handles almost all background events, and so these events will be *true* $\omega$ events, but contribute in total a flat distribution to the angular distributions
* Uses all 4 polarization orientations

The previous notebook (no output) [can be found here](./constrained_ratio.ipynb), and the presentation + complete notebook is [available online](https://halldweb.jlab.org/doc-private/DocDB/ShowDocument?docid=7027). A crucial difference in the results between these two is that the previous study utilized 100 bootstrap fits per bin to better approximate the errors on the parameters. To keep this comparison study fast, we do no bootstrapping, and so all errors are MINUIT only. Previous studies have shown our MINUIT errors tend to be roughly 1/2 the size of the bootstrap errors.

The details of the fit can be described as:
* Split data into bins of mass and $-t$
  * 20 MeV wide bins from $1.0 < M_{\omega\pi^0} < 2.0$ 
  * Focus only on the $0.1 < -t < 0.2$ range for now
* Perform 500 randomized fits in each bin of mass and $-t$ independently 

## Setup


In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/data-fits/fsroot-check/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
# load in the preprocessed results 
with open(f"{STUDY_DIR}/t_0.1-0.2/preprocessed_results.pkl", "rb") as f:
    data = pkl.load(f)
    new_results = ResultManager(**data)

with open(f"{STUDY_DIR}/../spin-1-constrained-ratio/t_0.1-0.2/preprocessed_results.pkl", "rb") as f:
    data = pkl.load(f)
    old_results = ResultManager(**data)


In [ ]:
new_results.summary()

In [ ]:
old_results.summary()

## Analysis

### Standard Plots
First step is to print out all the standard plots to view the model's performance

In [ ]:
utils.big_print(f"t bin: 0.1-0.2", 2.0)
for pdf in pathlib.Path(f"{STUDY_DIR}/t_0.1-0.2/plots").glob("*.pdf"):
    if pdf.name in ["jp.pdf", "waves.pdf", "matrix.pdf", "proj_moments.pdf"]:
        utils.display_pdf(str(pdf), 0)

In [ ]:
utils.big_print(f"t bin: 0.1-0.2", 2.0)
for pdf in pathlib.Path(f"{STUDY_DIR}/../spin-1-constrained-ratio/t_0.1-0.2/plots").glob("*.pdf"):
    if pdf.name in ["jp.pdf", "waves.pdf", "matrix.pdf", "proj_moments.pdf"]:
        utils.display_pdf(str(pdf), 0)

### D/S Ratio
Compare the D/S Ratio to E852 results, and check for correlations using the bootstrap fits

In [ ]:
plt.rcParams["figure.dpi"] = 300
new_results.plot.diagnostic.ds_ratio()

In [ ]:
# The result class does not yet have the correlation matrix loaded, but we do have the dataframe to access it
corr_df = pd.read_csv(f"{STUDY_DIR}/t_0.1-0.2/raw/best_corr.csv")
ratio_corrs = corr_df[corr_df["parameter"] == "dsratio"]["dphase"]

In [ ]:
mass = new_results.plot.intensity._masses
fig, ax = plt.subplots(figsize=(12,3))
plt.plot(mass, ratio_corrs, marker=".", linestyle="none", color="black")
plt.xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")
plt.ylabel(r"$\rho(D/S, D-S)$", loc="top")
plt.ylim(-1, 1)

In [ ]:
plt.rcParams["figure.dpi"] = 300
old_results.plot.diagnostic.ds_ratio()

### b1 - Vector Interaction

In [ ]:
new_results.plot.phase.mass_phase("p1p0S", "p1mpP", {"color": "tab:blue"}, {"color": "tab:orange"})

In [ ]:
old_results.plot.phase.mass_phase("p1p0S", "p1mpP", {"color": "tab:blue"}, {"color": "tab:orange"})

### Fit Fraction Comparison
Since our statistics have changed with the new event selection, we can only directly compare the results by their fit fraction (wave intensity / total intensity). Let's compare the coherent sums and wave fit fractions here